In [ ]:
from matplotlib import pyplot as plt
import cv2
import os

In [ ]:
# Fonction pour saisir le chemin de l'image à chercher
def chemin_image_chercher(): 
    print("Entrez votre chemin de l'image du Pokémon à chercher : ")
    path = input()
    return path


In [ ]:

# Fonction pour saisir le chemin de l'image dans laquelle chercher
def chemin_image_ou_chercher():
    print("Entrez votre chemin de l'image dans laquelle chercher le Pokémon : ")
    path = input()
    return path

In [ ]:
# Vérification de la validité du chemin de l'image
def verif_image(path):  
    return os.path.exists(path)

In [ ]:
# Fonction pour convertir une image en niveaux de gris
def NiveauGris(image):
    imgNG = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return imgNG

In [ ]:
# Fonction pour redimensionner une image
def redim(imageBase, largeur, hauteur):
    imgR = cv2.resize(imageBase, (largeur, hauteur), interpolation=cv2.INTER_AREA)
    return imgR

In [ ]:
# Fonction pour charger l'image et vérifier sa validité
def charger_image(chemin):
    if verif_image(chemin): 
        image = cv2.imread(chemin, cv2.IMREAD_COLOR)
        return image
    else:
        print("L'image n'est pas présente dans le chemin donné ou l'extension de l'image n'est pas bonne.")
        return None

In [ ]:
# Sélection du Pokémon à chercher et de l'image cible
chemin_pokemon = chemin_image_chercher()
chemin_image_grande = chemin_image_ou_chercher()

In [ ]:
# Chargement des images
pokemon_image = charger_image(chemin_pokemon)
grande_image = charger_image(chemin_image_grande)


In [ ]:
# Vérifier si les deux images sont valides
if pokemon_image is None or grande_image is None:
    exit()


In [ ]:
# Conversion en niveaux de gris
pokemon_img_ng = NiveauGris(pokemon_image)
grande_img_ng = NiveauGris(grande_image)

In [ ]:
# Redimensionnement des images pour accélérer le processus
pokemon_img_resized = redim(pokemon_img_ng, 800, 800)
grande_img_resized = redim(grande_img_ng, 800, 800)

In [ ]:
# Création de l'objet ORB pour détecter et calculer les points-clés
orb = cv2.ORB_create()

In [ ]:
# Détection des points-clés et des descripteurs pour les deux images
kp_pokemon, des_pokemon = orb.detectAndCompute(pokemon_img_resized, None)
kp_grande, des_grande = orb.detectAndCompute(grande_img_resized, None)

In [ ]:

# Utilisation de la méthode des k plus proches voisins avec k=2
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
matches = bf.knnMatch(des_pokemon, des_grande, k=2)

In [ ]:
# Application de la règle du ratio pour filtrer les bons matches
good_matches = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:  # Seulement les bonnes correspondances
        good_matches.append(m)

In [ ]:
# Récupération des meilleures correspondances (par exemple, top 25)
good_matches = sorted(good_matches, key=lambda x: x.distance)

In [ ]:
# Dessin des matches
img_matches = cv2.drawMatches(
    pokemon_image, kp_pokemon, grande_image, kp_grande, good_matches[:25], grande_image,
    flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

In [ ]:
# Affichage des résultats
plt.imshow(img_matches)
plt.title('Résultats de la recherche de Pokémon')
plt.show()

In [ ]:
# Ajout des repères visuels sur les zones où le Pokémon est trouvé
for match in good_matches[:10]:  # Utiliser les meilleures correspondances pour le marquage
    pokemon_pt = kp_pokemon[match.queryIdx].pt
    grande_pt = kp_grande[match.trainIdx].pt
    cv2.circle(grande_image, (int(grande_pt[0]), int(grande_pt[1])), 5, (0, 255, 0), 2)

In [ ]:
# Affichage de l'image finale avec les repères visuels
plt.imshow(cv2.cvtColor(grande_image, cv2.COLOR_BGR2RGB))
plt.title('Pokémon détecté avec repères visuels')
plt.show()